# <div align="center"> CV Api Test </div>

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from k12libs.utils.nb_easy import k12ai_post_request
from k12libs.utils.nb_easy import k12ai_post_cv_request
from k12libs.utils.nb_easy import k12ai_get_data
from k12libs.utils.nb_easy import k12ai_del_data
from k12libs.utils.nb_easy import k12ai_out_data
from k12libs.utils.nb_easy import k12ai_print
from k12libs.utils.nb_easy import k12ai_count_down
from k12libs.utils.nb_widget import k12ai_schema_parse

import json

In [6]:
schema_mnist = k12ai_post_request(
    uri='k12ai/framework/schema',
    data={
        'service_name': 'k12cv',
        'service_task': 'cls',
        'dataset_name': 'cifar10',
    }
)
schema_json = json.loads(schema_mnist)['data']
# k12ai_print(schema_json)
print(schema_json)

{"objs":[{"objs":[{"name":{"cn":"Data","en":"Data"},"objs":[{"objs":[{"_id_":"task","default":"cls","name":{"cn":"Task","en":"Task"},"readonly":true,"type":"string"},{"_id_":"method","default":"image_classifier","name":{"cn":"Method","en":"Method"},"readonly":true,"type":"string"},{"_id_":"data.include_val","default":false,"name":{"cn":"Include Val","en":"Include Val"},"type":"bool"}],"type":"H"},{"objs":[{"_id_":"dataset","default":"default","name":{"cn":"Loader","en":"Loader"},"readonly":true,"type":"string"},{"_id_":"data.workers","default":4,"max":8,"min":1,"name":{"cn":"Workers","en":"Workers"},"type":"int"},{"_id_":"data.drop_last","default":false,"name":{"cn":"Drop Last","en":"Drop Last"},"type":"bool"}],"type":"H"},{"objs":[{"_id_":"data.image_tool","default":"pil","name":{"cn":"Image Tool","en":"Image Tool"},"objs":[{"name":{"cn":"PIL","en":"PIL"},"value":"pil"},{"name":{"cn":"CV2","en":"CV2"},"value":"cv2"}],"type":"string-enum"},{"_id_":"data.input_mode","default":"RGB","nam

In [7]:
generator = k12ai_schema_parse(json.loads(schema_json), lan='cn', debug=False);

Box(children=(Box(children=(VBox(children=(Tab(children=(VBox(children=(HBox(children=(Text(value='cls', conti…

In [8]:
context = k12ai_post_cv_request(
    uri='k12ai/framework/execute',
    op='train.stop',
    user='10000001',
    uuid='10000001',
)
k12ai_del_data(context['key'])

'success'

In [9]:
context = k12ai_post_cv_request(
    uri='k12ai/framework/execute',
    op='train.start',
    user='10000001',
    uuid='10000001',
    params=generator.get_all_kv()
)
k12ai_del_data(context['key'])

'success'

In [50]:
k12ai_out_data(context['key'], ['status', 'metrics', 'error'], hook_func=k12ai_count_down(10))